<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>



We start by Downloading all the dependencies that we will need:

In [4]:
# import requests and BeautifulSoup to parse the wikipedia page
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

The next step would be to get the page content and store it in a BeautifulSoup object.

Note that i used **lxml** as the html processing library.

In [8]:
# wikipedia page link
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# get the html source code of the postal codes page
html_source = requests.get(url).text

# create a BeautifulSoup object to parse the html code
soup_page = BeautifulSoup(html_source,'lxml')

In this section, we will parse the BeautifulSoup object, grab the html table and convert it into a dataframe:

In [13]:
# parse the object to grab the html table
table = soup_page.find('div', class_='mw-body').find('div',id='bodyContent').find('div',id='mw-content-text').div.table.tbody

# initialize an empty array to store the columns names
columns = []

# parse the columns names
for column in table.find('tr').find_all('th'):
  columns.append(str(column.text).splitlines()[0])

# initialize the dataframe with the given columns
df = pd.DataFrame(columns=columns)

# iterate over the html table and append the rows and columns to the dataframe
for tr in table.find_all('tr')[1:]:
    tds = tr.find_all('td')
    row = [ str(td.text).splitlines()[0] for td in tds ]
    df = df.append({'Postcode': row[0],
               'Borough':row[1], 
               'Neighbourhood': row[2]}, ignore_index=True)

df.columns = ['PostalCode', 'Borough', 'Neighbourhood']


We start cleaning the data by:
    1.  Ignoring the cells with a borough that is Not assigned
    2.  Setting the neighborhood as the borough if it is Not assigned

In [14]:
df = df.loc[df['Borough']!= 'Not assigned']

df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])

df = df.reset_index(drop=True)

   3. group all the neighbourhouds with the same Postcode in one row, seperated with commas.

In [15]:
df = df.astype(str).groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

The last step would be to print the number of rows of the final dataframe:

In [16]:
df.shape[0]

103

## Part2: get the geographical coordinates of the neighborhoods in the Toronto

I'm using the csv file for that purpose instead of the Geocoder package because i wasn't able to get the data with it:

In [23]:
# Load the csv file into a dataframe and update the columns names

gc_df = pd.read_csv('http://cocl.us/Geospatial_data')
gc_df.columns = ['PostalCode', 'Latitude', 'Longitude']

Now we need to merge our neighbourhoods dataframe with the 

In [18]:
ndf = df.merge(gc_df, on='PostalCode')

In [21]:
ndf.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
